# Dryad API

# Setup

## Instructions

This notebook utilizes the Data Dryad API. No API Token is required.

## Additional Information

Documentation Guide:
- Dryad API ([Dryad](https://datadryad.org/api/v2/docs/))
- Dryad API ([GitHub](https://github.com/CDL-Dryad/dryad-app/tree/main/documentation/apis))

## Imports

In [1]:
import requests # For querying data from API
import pandas as pd # For storing/manipulating query data
from tqdm import tqdm # Gives status bar on loop completion
from collections import OrderedDict
from flatten_json import flatten

# For loading credentials
import pickle
import os

# Data Wrangling

## Setup

In [2]:
# Search constants
BASE_URL = 'https://datadryad.org/api/v2'

## Extracting

In [3]:
def get_all_search_outputs(search_terms, flatten_output=False):
    """
    Call the Data Dryad API for each search term. 
    Results are retured in results['({term},)'] = df
    
    Params:
    - search_terms : list-like 
        collection of search terms to query over
    - flatten_output : bool, optional (default=False)
        flag for flattening nested columns of output
    
    Returns:
    - results : dict
        dictionary consisting of returned DataFrames from get_search_output for each query
    """
    
    results = OrderedDict()

    for search_term in search_terms:
        results[(search_term,)] = get_individual_search_output(search_term, flatten_output)
        
    return results

In [4]:
def _conduct_search_over_pages(search_url, search_params, flatten_output=False, delim=None):
    # Make sure proper delim passed in
    if delim:
        assert isinstance(delim, str), 'Incorrect delim parameter passed in. Must be of type str'
    
    search_df = pd.DataFrame()
    
    # Perform initial search & convert results to json
    response = requests.get(search_url, params=search_params)
    output = response.json()
    
    # Loops over the search as long as the page was not empty
    while output.get('count'):
        # Extract relevant output data
        output = output['_embedded']
        
        if delim:
            output = output[delim]
        
        # Flatten output if necessary
        if flatten_output:
            output = [flatten(result) for result in output]
        else:
            output = output
        
        output_df = pd.DataFrame(output)
        output_df['page'] = search_params['page']
        
        search_df = pd.concat([search_df, output_df]).reset_index(drop=True)
        
        # Increment the page number to perform another search
        search_params['page'] += 1
        
        # Perform next search and convert results to json
        response = requests.get(search_url, params=search_params)
        output = response.json()

    return search_df

In [5]:
def get_individual_search_output(search_term, flatten_output=False):
    """
    Returns a list of all datasets available from the Data Dryad API
    
    Params:
    - search_term : str
    - flatten_output : bool, optional (default=False)
        flag for flattening nested columns of output
        
    Returns:
    - pandas.DataFrame
        DataFrame containing the output of the search query
    """
    
    # Set search params
    search_url = f'{BASE_URL}/search'
    start_page = 1
    page_size = 100
    
    search_params = {
        'q': search_term,
        'page': start_page,
        'per_page': page_size
    }
    
    return _conduct_search_over_pages(search_url, search_params, flatten_output, delim='stash:datasets')

### Run initial API query functions

In [6]:
search_terms = ['machine learning']

In [7]:
search_output_dict = get_all_search_outputs(search_terms, flatten_output=True)

#### Take a look at the results

In [8]:
search_output_dict[('machine learning',)].head()

,_links_self_href,_links_stash:versions_href,_links_stash:version_href,_links_stash:download_href,_links_curies_0_name,_links_curies_0_href,_links_curies_0_templated,identifier,id,storageSize,...,authors_30_lastName,authors_30_email,authors_30_affiliation,authors_30_affiliationROR,authors_31_firstName,authors_31_lastName,authors_31_email,authors_31_affiliation,authors_31_affiliationROR,authors_9_orcid
0,/api/v2/datasets/doi%3A10.5061%2Fdryad.nj2mg77,/api/v2/datasets/doi%3A10.5061%2Fdryad.nj2mg77...,/api/v2/versions/29027,/api/v2/datasets/doi%3A10.5061%2Fdryad.nj2mg77...,stash,https://github.com/CDL-Dryad/stash/blob/main/s...,true,doi:10.5061/dryad.nj2mg77,28194,1769456,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,/api/v2/datasets/doi%3A10.5061%2Fdryad.q6ft5,/api/v2/datasets/doi%3A10.5061%2Fdryad.q6ft5/v...,/api/v2/versions/17751,/api/v2/datasets/doi%3A10.5061%2Fdryad.q6ft5/d...,stash,https://github.com/CDL-Dryad/stash/blob/main/s...,true,doi:10.5061/dryad.q6ft5,17689,31656,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,/api/v2/datasets/doi%3A10.7280%2FD1WQ3R,/api/v2/datasets/doi%3A10.7280%2FD1WQ3R/versions,/api/v2/versions/118084,/api/v2/datasets/doi%3A10.7280%2FD1WQ3R/download,stash,https://github.com/CDL-Dryad/stash/blob/main/s...,true,doi:10.7280/D1WQ3R,67581,241082358,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,/api/v2/datasets/doi%3A10.6075%2FJ03J3BGF,/api/v2/datasets/doi%3A10.6075%2FJ03J3BGF/vers...,/api/v2/versions/108192,/api/v2/datasets/doi%3A10.6075%2FJ03J3BGF/down...,stash,https://github.com/CDL-Dryad/stash/blob/main/s...,true,doi:10.6075/J03J3BGF,50040,240410779544,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,/api/v2/datasets/doi%3A10.5061%2Fdryad.xksn02vbq,/api/v2/datasets/doi%3A10.5061%2Fdryad.xksn02v...,/api/v2/versions/55637,/api/v2/datasets/doi%3A10.5061%2Fdryad.xksn02v...,stash,https://github.com/CDL-Dryad/stash/blob/main/s...,true,doi:10.5061/dryad.xksn02vbq,40246,141267,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Metadata

In [9]:
def get_query_metadata(object_paths, flatten_output=False):
    """
    Retrieves the metadata for the file/files listed in object_paths
    
    Params:
    - object_paths : str/list-like
        string or list of strings containing the paths for the objects
    - flatten_output : bool, optional (default=False)
        flag for flattening nested columns of output
    
    Returns:
    - metadata_df : pandas.DataFrame
        DataFrame containing metadata for the requested dataset
    """
    
    # If a singular search term is provided as a string, need to wrap it in a list
    if type(object_paths) == str:
        object_paths = [object_paths]
    
    # Make sure our input is valid
    assert len(object_paths) > 0, 'Please enter at least one object id'
    
    # Set search variables
    start_page = 1
    metadata_df = pd.DataFrame()
    
    # Request the metadata for each object
    for object_path in tqdm(object_paths):
        # Set search variables
        search_url = f'{BASE_URL}/versions/{object_path}/files'
        search_params = {'page': start_page}
        
        # Conduct search
        object_df = _conduct_search_over_pages(search_url, search_params, flatten_output, delim='stash:files')

        # Add relevant data to DataFrame and merge
        object_df['id'] = object_path
        object_df['page'] = search_params['page']
        metadata_df = pd.concat([metadata_df, object_df]).reset_index(drop=True)
    
    return metadata_df

In [10]:
def get_all_metadata(search_output_dict, flatten_output=False):
    """
    Retrieves all of the metadata that relates to the provided DataFrames
    
    Params:
    - search_output_dict : dict
        Dictionary of DataFrames from get_all_search_outputs
    - flatten_output : bool, optional (default=False)
        flag for flattening nested columns of output  
      
    Returns:
    - metadata_dict : collections.OrderedDict
        OrderedDict of DataFrames with metadata for each query
        Order matches the order of search_output_dict
    """
    
    ## Extract IDs from DataFrame, and returns as list of strings
    metadata_dict = OrderedDict()

    for query, df in search_output_dict.items():
        search_term = query[0]
        print(f'Retrieving {search_term} metadata')
        
        # Create object paths
        object_paths = df.id.convert_dtypes(convert_string=True).tolist()

        metadata_dict[query] = get_query_metadata(object_paths, flatten_output)
    
    return metadata_dict

### Run metadata API query functions

In [11]:
metadata_dict = get_all_metadata(search_output_dict, flatten_output=True)

  0%|          | 0/237 [00:00<?, ?it/s]

Retrieving machine learning metadata


100%|██████████| 237/237 [02:13<00:00,  1.78it/s]


#### Take a look at the results

In [12]:
metadata_dict[('machine learning',)].head()

,_links_self_href,_links_stash:dataset_href,_links_stash:version_href,_links_stash:files_href,_links_stash:file-download_href,_links_stash:download_href,_links_curies_0_name,_links_curies_0_href,_links_curies_0_templated,path,url,size,mimeType,status,digest,digestType,description,page,id
0,/api/v2/files/105376,/api/v2/datasets/doi%3A10.5061%2Fdryad.61pm78v,/api/v2/versions/28194,/api/v2/versions/28194/files,/api/v2/files/105376/download,/api/v2/files/105376/download,stash,https://github.com/CDL-Dryad/stash/blob/main/s...,true,Datas-ZLL.xlsx,https://dryad-assetstore-east.s3.amazonaws.com...,39464.0,application/vnd.openxmlformats-officedocument....,created,eb69ba9befabb98fb4cbc75ab6eb23ca,md5,"Carbon concentration, carbon stock, and distri...",2,28194
1,/api/v2/files/59371,/api/v2/datasets/doi%3A10.5061%2Fdryad.48vb7,/api/v2/versions/17689,/api/v2/versions/17689/files,/api/v2/files/59371/download,/api/v2/files/59371/download,stash,https://github.com/CDL-Dryad/stash/blob/main/s...,true,GundersonLeal15_AmNat_Anole_Data.txt,https://dryad-assetstore-east.s3.amazonaws.com...,11605.0,text/plain,created,7f10d45c751b82f8e05f2932e221b7a0,md5,Activity data for Anolis cristatellus,2,17689
2,/api/v2/files/59372,/api/v2/datasets/doi%3A10.5061%2Fdryad.48vb7,/api/v2/versions/17689,/api/v2/versions/17689/files,/api/v2/files/59372/download,/api/v2/files/59372/download,stash,https://github.com/CDL-Dryad/stash/blob/main/s...,true,Christian varanid spp activity.csv,https://dryad-assetstore-east.s3.amazonaws.com...,1266.0,text/csv,created,6e44028c2e001c24d33427d172eb893d,md5,Data on activity for Varanids from Christian a...,2,17689
3,/api/v2/files/232146,/api/v2/datasets/doi%3A10.5061%2Fdryad.3xsj3txbt,/api/v2/versions/50040,/api/v2/versions/50040/files,/api/v2/files/232146/download,/api/v2/files/232146/download,stash,https://github.com/CDL-Dryad/stash/blob/main/s...,true,jags_code.txt,NaN,3219.0,text/plain,copied,NaN,NaN,NaN,2,50040
4,/api/v2/files/232147,/api/v2/datasets/doi%3A10.5061%2Fdryad.3xsj3txbt,/api/v2/versions/50040,/api/v2/versions/50040/files,/api/v2/files/232147/download,/api/v2/files/232147/download,stash,https://github.com/CDL-Dryad/stash/blob/main/s...,true,rcode_for_running_model.R,NaN,1278.0,application/octet-stream,copied,NaN,NaN,NaN,2,50040
